In [1]:
import pandas as pd
import numpy as np
import os
import csv
import datetime as d
from datetime import datetime as D
import matplotlib.ticker as ticker
import random 
import time
import math
import scipy
from scipy import stats
import gzip
import sys
import datetime
import ast

In [36]:
data_path = '../../../../../../../Volumes/Seagate_Qing/Safegraph_clean/'
# save_dir = '../../../../../../../Volumes/Seagate_Qing/Safegraph_clean/'
years = ['2020','2021','2022']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
#np.save('ym_datesofweek',ym_datesofweek)
ym_datesofweek = np.load('../data/ym_datesofweek.npy',allow_pickle=True).item()

In [3]:
US_states = pd.read_csv('us-state-ansi-fips.csv')
state_lists_0 = US_states.stusps.values
state_lists = [s[1:] for s in state_lists_0]
state_lists.remove('HI')
state_lists.remove('AK') ### remove the non mainland

real_data_path = '/Users/qingyao/OneDrive - cumc.columbia.edu/machine_learning_data/'
fips_main = pd.read_csv(real_data_path+'county_data/fips_mainland.csv',names=['FIPS'])
fips_main_sorted = fips_main.sort_values(by='FIPS')
fips_main_sorted['poi_idx'] = fips_main_sorted.index

In [4]:
csv.field_size_limit(500 * 1024 * 1024)

131072

In [5]:
df_header = pd.read_csv('header.csv')
df_header.iloc[[32,34,36,38],0].values

array(['raw_visit_counts', 'visits_by_day', 'poi_cbg',
       'visitor_home_aggregation'], dtype=object)

In [6]:
selected_header = ['raw_visit_counts', 'visits_by_day', 'poi_cbg','visitor_home_aggregation']

In [38]:
%run ../codes/mobility_matrix_extraction.py

In [50]:
y = '2020'
for m in ['04']:
    print('month:', m)
    week_start_dates = ym_datesofweek[y + m]
    for start_day in week_start_dates[:1]:
        print(start_day)
#         M_w = np.zeros((7,3108,3108)) #M_weekly = np.zeros((7,3108,3108)) ## 
        for i in range(1,2):  ### there are 40 files !!!
            print(i)
            new_name = data_path + y + '_' + m + '_' + start_day + '_{}.csv'.format(i)
            df_file = pd.read_csv(new_name)
            df_noca = df_file[~df_file['poi_cbg'].astype(str).str.contains('CA:')]
            df_noca['poi_fips'] = df_noca['poi_cbg'].astype(str).str[:-7].astype(int)
            df_i = pd.merge(df_noca,fips_main_sorted,left_on='poi_fips',right_on='FIPS',how='left') 
            df_i_main = df_i.dropna()
            
            df_m = pd.DataFrame()
            df_m[['raw_visit_counts','poi_idx']] = df_i_main[['raw_visit_counts','poi_idx']]
            df_m['visits_by_day_list'] = df_i_main['visits_by_day'].apply(lambda x: np.array(ast.literal_eval(x)))
            df_m['multiplier'] = df_m.apply(lambda x: x['visits_by_day_list']/x['raw_visit_counts'], axis=1) ### multiplier
            df_m['visitor_home_list'] = df_i_main['visitor_home_aggregation'].apply(lambda x: x[1:-1].replace('"','').split(','))
            df_m[['home_idx_list', 'modified_visit_list']] = df_m.apply(lambda x: process_visitor_data(x['visitor_home_list'],x['raw_visit_counts'],fips_main_sorted),axis=1)
            df_m['vis_daily_matrix'] = df_m.apply(lambda x: dayvisits_by_visitor(x['modified_visit_list'], x['multiplier']),axis=1)
#             df_i_main.loc[:,'visits_by_day_list'] = df_i_main['visits_by_day'].apply(lambda x: np.array(ast.literal_eval(x)))
#             df_i_main.loc[:,'multiplier'] = df_i_main.apply(lambda x: x['visits_by_day_list']/x['raw_visit_counts'], axis=1) ### multiplier
#             df_i_main.loc[:,'visitor_home_list'] = df_i_main['visitor_home_aggregation'].apply(lambda x: x[1:-1].replace('"','').split(','))
            
#             df_i_main.loc[:,['home_idx_list', 'modified_visit_list']] = df_i_main.apply(lambda x: process_visitor_data(x['visitor_home_list'],x['raw_visit_counts'],fips_main_sorted),axis=1)
#             df_i_main.loc[:,'vis_daily_matrix'] = df_i_main.apply(lambda x: dayvisits_by_visitor(x['modified_visit_list'], x['multiplier']),axis=1)
            for j_data_idx in range(len(df_m)):
                df_j = df_m.iloc[j_data_idx]
                M_w = mobility_extract_per_poi(df_j, M_w)

month: 04
06
1


In [48]:
df_i_main.head()

,raw_visit_counts,visits_by_day,poi_cbg,visitor_home_aggregation,poi_fips,FIPS,poi_idx
0,35,"[4,5,6,6,5,7,2]",391130404013,"{""39113040401"":6,""39165030200"":4,""39165030102""...",39113,39113,2065
1,70,"[5,7,9,15,8,16,10]",450790116081,"{""45079011607"":9,""45079011606"":6,""45079011608""...",45079,45079,2321
2,7,"[1,1,0,2,3,0,0]",370670001001,"{""37169070503"":4}",37067,37067,1889
3,5,"[0,0,1,0,0,3,1]",421010356012,"{""42101034502"":4}",42101,42101,2260
4,66,"[9,19,13,9,7,9,0]",190990407001,"{""19099040700"":13,""19099040200"":5,""19083480100...",19099,19099,804


In [51]:
df_m.head()

,raw_visit_counts,poi_idx,visits_by_day_list,multiplier,visitor_home_list,home_idx_list,modified_visit_list,vis_daily_matrix
0,35,2065,"[4, 5, 6, 6, 5, 7, 2]","[0.11428571428571428, 0.14285714285714285, 0.1...","[39113040401:6, 39165030200:4, 39165030102:4, ...","[2065, 2091, 2091, 2065]","[11.666666666666666, 7.777777777777778, 7.7777...","[[1.3333333333333333, 1.6666666666666665, 2.0,..."
1,70,2321,"[5, 7, 9, 15, 8, 16, 10]","[0.07142857142857142, 0.1, 0.12857142857142856...","[45079011607:9, 45079011606:6, 45079011608:6, ...","[2321, 2321, 2321, 2321, 2321, 2321, 2313, 232...","[11.454545454545455, 7.636363636363637, 7.6363...","[[0.8181818181818181, 1.1454545454545455, 1.47..."
2,7,1889,"[1, 1, 0, 2, 3, 0, 0]","[0.14285714285714285, 0.14285714285714285, 0.0...",[37169070503:4],[1940],[7.0],"[[1.0, 1.0, 0.0, 2.0, 3.0, 0.0, 0.0]]"
3,5,2260,"[0, 0, 1, 0, 0, 3, 1]","[0.0, 0.0, 0.2, 0.0, 0.0, 0.6, 0.2]",[42101034502:4],[2260],[5.0],"[[0.0, 0.0, 1.0, 0.0, 0.0, 3.0, 1.0]]"
4,66,804,"[9, 19, 13, 9, 7, 9, 0]","[0.13636363636363635, 0.2878787878787879, 0.19...","[19099040700:13, 19099040200:5, 19083480100:4,...","[804, 804, 796, 831, 831, 831]","[25.235294117647058, 9.705882352941178, 7.7647...","[[3.441176470588235, 7.264705882352941, 4.9705..."


In [33]:
#normalize by  people num
# M_norm = M_w / pop[np.newaxis, :, np.newaxis]
save_dir = '/Users/qingyao/Documents/branching_data/'
for d in range(7):
    np.savetxt(save_dir+'M_{}{}_{}.csv'.format(y,m,(int(start_day)+d)), M_w[d], delimiter=",")